# Full Walkthrough: Delivery 3

Full walkthrough of automated data gathering, data transformations, calculation of filtering and doubling rate and in the end development of dynamic dashboard depicting COVID-19 data of all the countries.  
  
  Name: Aditya Sanjay Landge  
  Matrikel Nr: 414705

In [1]:
## check some parameters
## depending where you launch your notebook, the relative path might not work
## you should start the notebook server from your base path
## when opening the notebook, typically your path will be ../ads_covid-19/notebooks
import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: EDS_ADS_COVID-19-master'

# 1. Get the updated data: Clone data from GitHub

In [3]:
# %load src/data/get_data.py

import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

def get_johns_hopkins():
    
    ''' Get data by a git clone request, the source code has to be cloned/pulled first
        Result is stored in the predifined csv structure
    '''
    
    git_repo = 'https://github.com/CSSEGISandData/COVID-19.git'
    path = "data/raw/COVID-19/"
  

    git_clone = subprocess.Popen( "git clone "+git_repo,
                         cwd = os.path.dirname(os.path.realpath(path) ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )                
                    
    (out, error) = git_clone.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))


def get_current_data_germany():
    ''' Get current data from germany, attention API endpoint not too stable
        Result data frame is stored as pd.DataFrame

    '''
    # 16 states
    #data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    # 400 regions / Landkreise
    data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    json_object=json.loads(data.content)
    full_list=[]
    for pos,each_dict in enumerate (json_object['features'][:]):
        full_list.append(each_dict['attributes'])

    pd_full_list=pd.DataFrame(full_list)
    pd_full_list.to_csv('data/raw/NPGEO/GER_state_data.csv')
    print(' Number of regions rows: '+str(pd_full_list.shape[0]))

if __name__ == '__main__':
    get_johns_hopkins()
    get_current_data_germany()


Error : b"fatal: destination path 'COVID-19' already exists and is not an empty directory.\n"
out : b''
 Number of regions rows: 411


# 2. Data Transformation

In [4]:
# %load src/data/process_JH_data.py
import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data():
    ''' 
        Transformes the COVID data in a relational data set
    '''

    data_path='data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'Country',
                      'Province/State':'State'})

    pd_data_base['State']=pd_data_base['State'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model=pd_data_base.set_index(['State','Country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'Date',
                                                   0:'Confirmed'},
                                                  )

    pd_relational_model['Date']=pd_relational_model.Date.astype('datetime64[ns]')
    
    #US data processing
    data_path='data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
    pd_raw_US=pd.read_csv(data_path)
    
    # drop all non relevant fields! 

    pd_raw_US=pd_raw_US.drop(['UID', 'iso2', 'iso3', 'code3', 'Country_Region','FIPS', 'Admin2',  'Lat', 'Long_', 'Combined_Key'],axis=1)
    pd_data_base_US=pd_raw_US.rename(columns={'Province_State':'State'}).copy()
    
    pd_relational_model_US=pd_data_base_US.set_index(['State']) \
                                .T                              \
                                .stack()             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'Date',
                                                   0:'Confirmed'},
                                                  )
    pd_relational_model_US['Country']='US'
    pd_relational_model_US['Date']=[datetime.strptime( each,"%m/%d/%y") for each in pd_relational_model_US.Date] # convert to datetime
    
    #Swap Columns 'Confirmed' and 'Country'
    pd_relational_model_US.shape[1]
    pd_relational_model_US['temp']=pd_relational_model_US['Confirmed']
    pd_relational_model_US['Confirmed']=pd_relational_model_US['Country']
    pd_relational_model_US['Country']=pd_relational_model_US['temp']
    pd_relational_model_US.drop(columns =['temp'], inplace=True)
    pd_relational_model_US=pd_relational_model_US.rename(columns={'Confirmed':'Country','Country':'Confirmed'})

    # update the large relational file
    pd_relational_model_all=pd_relational_model[pd_relational_model['Country']!='US'].reset_index(drop=True)
    pd_relational_model_all=pd.concat([pd_relational_model_all,pd_relational_model_US],ignore_index=True)
    pd_relational_model_all.to_csv('data/processed/20220719_COVID_relational_confirmed.csv',index=False)
    
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    print(' Latest date is: '+str(max(pd_relational_model.Date)))
    
if __name__ == '__main__':
    store_relational_JH_data()
    

 Number of rows stored: 260775
 Latest date is: 2022-07-24 00:00:00


# 3. Filter and Doubling Rate Calculation

In [5]:
# %load src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='Confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='Confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='Confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['State','Country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['State','Country',filter_on]].groupby(['State','Country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='Confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['State','Country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['State','Country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('data/processed/20220719_COVID_relational_confirmed.csv',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('Date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'Confirmed_filtered')


    mask=pd_result_larg['Confirmed']>100
    pd_result_larg['Confirmed_filtered_DR']=pd_result_larg['Confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('data/processed/COVID_final_set.csv',index=False)
    print(pd_result_larg[pd_result_larg['Country']=='India'].tail())


the test slope is: [2.]
              Date State Country   Confirmed  Confirmed_filtered  \
3210730 2022-07-20    no   India  43824728.0          43825001.8   
3210731 2022-07-21    no   India  43847065.0          43846140.4   
3210732 2022-07-22    no   India  43868476.0          43866929.0   
3210733 2022-07-23    no   India  43888755.0          43887276.6   
3210734 2022-07-24    no   India  43905621.0          43907624.2   

         Confirmed_DR  Confirmed_filtered_DR  
3210730   2102.585129            2219.014012  
3210731   1931.147259            2120.904906  
3210732   2004.514782            2091.531213  
3210733   2104.490221            2132.758106  
3210734   2363.043065            2156.877302  


In [6]:
df_input_large=pd.read_csv('data/processed/COVID_final_set.csv')

# 4. Development of a Visual Board

In [ ]:
# %load src/visualization/visualize.py
import pandas as pd
import numpy as np

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())

df_input_large=pd.read_csv('data/processed/COVID_final_set.csv')


fig = go.Figure()

app = dash.Dash()

app.layout = html.Div([

    dcc.Markdown('''
    #  COVID-19 data visualisation dynamic dashboard

    Goal of the project is gather updated data of COVID-19 across the world from github,
    transform the data for our use, filter the data and calculate the doubling rate. In 
    the end, create a visualisation showing all these parameters of all the countries

    '''),

    dcc.Markdown('''
    ## Select country/countries whose data has to be visualised
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['Country'].unique()],
        value=['India', 'Germany','South Africa'], # countries which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 'Confirmed'},
        {'label': 'Timeline Confirmed Filtered', 'value': 'Confirmed_filtered'},
        {'label': 'Timeline Doubling Rate', 'value': 'Confirmed_DR'},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 'Confirmed_filtered_DR'},
    ],
    value='Confirmed',
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])



@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])

def update_figure(country_list,show_doubling):


    if 'DR' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['Country']==each]

        if show_doubling=='Confirmed_filtered_DR':
            df_plot=df_plot[['State','Country','Confirmed','Confirmed_filtered','Confirmed_DR','Confirmed_filtered_DR','Date']].groupby(['Country','Date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['State','Country','Confirmed','Confirmed_filtered','Confirmed_DR','Confirmed_filtered_DR','Date']].groupby(['Country','Date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.Date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                line_width=0.5,
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)


C:\Users\adity\AppData\Local\Temp\ipykernel_1832\3466964816.py:7: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\adity\AppData\Local\Temp\ipykernel_1832\3466964816.py:8: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


C:\Users\adity\Downloads\EDS_ADS_COVID-19-master\EDS_ADS_COVID-19-master
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
